In [2]:
import sqlite3
import torch
from transformers import AutoTokenizer, AutoModel

In [3]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()

C:\Users\Ogoun\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [4]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [6]:
conn = sqlite3.connect("videostat")
cur = conn.cursor()
cur.execute('SELECT video_id, title, description FROM VideoStatEntry')
rows = cur.fetchall()
conn.close()

In [7]:
print(len(rows))

1555687


In [16]:
statement = """CREATE TABLE IF NOT EXISTS vectors (
                video_id INTEGER PRIMARY KEY, 
                title TEXT, 
                description TEXT, 
                vector BLOB
        );"""

In [17]:
conn = sqlite3.connect("video_emb")
cursor = conn.cursor()
cursor.execute(statement)

In [18]:
sqlite_insert_with_param = """INSERT INTO vectors
                              (video_id, title, description, vector)
                              VALUES (?, ?, ?, ?);"""

In [19]:
for row in rows:
    video_id = row[0]
    title = row[1]
    description = row[2]
    encode_text = name + '. ' + description
    vector = embed_bert_cls(encode_text, model, tokenizer)
    
    data_tuple = (video_id, title, description, vector)
    cursor.execute(sqlite_insert_with_param, data_tuple)
    
conn.commit()
cursor.close()